# <font size="10">Custom entity recognition </font>
## Model environment setup

This notebook contains test code to train the implemented model in the generated training data outputted from
the [ner-train notebook located here](./ner-train-note.ipynb).

For simple loop model training (old), go [here](## Run training - using simple training loop from blank -- Old)

## Setup

In [27]:
!pip install -U spacy
!pip install spacy[lookups]


Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.3.2)
     |████████████████████████████████| 93.8MB 59kB/s 
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.3.2-py2.py3-none-any.whl size=93807573 sha256=219b3332a2f40191b7e27a829204135945318a57ba0a6858c3244e62b2ac4132
  Stored in directory: /root/.cache/pip/wheels/5b/f4/d0/bf720a06127c95d9be2a81d197a3f1998ee5fc63410944e28f
Successfully built spacy-lookups-data


In [2]:
!python -m spacy download en

!python -m spacy download en_core_web_md
!python -m spacy validate

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
✔ Loaded compatibility table

====================== Installed models (spaCy v2.3.2) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.3.1   ✔
package   en-core-web-md   en_core_web_md   2.3.1   ✔
link      en               en_core_web_sm   2.3.1   ✔



In [23]:
import pandas as pd
import random
import warnings
import json
import ast
import datetime as dt
from pathlib import Path
import os
import glob

##SpaCy

import en_core_web_sm
import en_core_web_md

import spacy
from spacy import displacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.pipeline import Sentencizer
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB
from spacy.util import minibatch, compounding
from spacy.util import decaying
from spacy.pipeline import Tagger
from spacy.pipeline import DependencyParser




# For Colab

In [4]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import training data

lets import the training data we generated:

In [5]:
train_path = r'/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/'

csvs = glob.glob(train_path + "/*.csv")


print('Filepath is :',(csvs))

train_list = []

for filename in csvs:
    df = pd.read_csv(filename, index_col=None, header=0)
    train_list.append(df)


Filepath is : ['/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset0.csv', '/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset1.csv', '/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/train/trainset2.csv']


In [6]:
DATA = pd.concat(train_list, axis=0, ignore_index=True)
DATA = DATA.drop_duplicates()
DATA[0:10]
print(len(DATA))

2101


In [7]:
#convert to list for model intake
TRAIN_DATA = DATA.values.tolist()

#for element in index 1 convert string (Entity position) to dictionary to be able to read by the model function
for position in TRAIN_DATA:
    position[1]=ast.literal_eval(position[1])
    
#Check our input list
print(TRAIN_DATA[0:2])

[['With increasing severe weather events and disasters \ntriggering greater numbers of costly power outages, there is a growing interest in generators for \nreliable backup power.', {'entities': [(161, 173, 'SUSTECH')]}], ['$3,153 \n\nNPC of Backup Power per Unit ($/kW)', {'entities': [(16, 28, 'SUSTECH')]}]]


## Run a test before training
### Test existing default spacy model 

In [8]:
nlp = en_core_web_md.load()
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f14bd681400>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f14bd822648>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f14bd822588>)]

In [9]:
doc = nlp('Here is a green roof on this house. A green roof is good.')
displacy.render(doc, style="ent")
# verified green roof does not match an entity in the NER

/usr/local/lib/python3.6/dist-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


'<div class="entities" style="line-height: 2.5; direction: ltr">Here is a green roof on this house. A green roof is good.</div>'

## TRAINING THE MODEL
## Train model setup

### Define compounding batch size

In [10]:
def get_batches(train_data, model_type):
    max_batch_sizes = {"tagger": 32, "parser": 16, "ner": 16, "textcat": 64}
    max_batch_size = max_batch_sizes[model_type]
    if len(train_data) < 1000:
        max_batch_size /= 2
    if len(train_data) < 500:
        max_batch_size /= 2
    batch_size = compounding(1, max_batch_size, 1.001)
    batches = minibatch(train_data, size=batch_size)
    return batches

In [11]:
#New training model loop to either accept existing model, if not model is not defined then create a blank nlp model using english vocab

def train_model(**model_params):

    model = model_params['model']
    iterations = model_params['iterations']
    train_data = model_params['train_data']
    dropout = model_params['dropout']
    
    
    random.seed(0)
    
    if model is not None:
        nlp = model.load() #load existing spacy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")
        #lang = "en"
        #cls = spacy.util.get_lang_class(lang)   # 1. Get Language instance, e.g. English()
        #nlp = cls() 
        print("Created blank 'en' model")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise,get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    #ner.add_label(LABEL)  # add new entity label to entity recognizer
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            #print(ent[2])

    # Adding extraneous labels shouldn't mess anything up
    ner.add_label("VEGETABLE")
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    # only train NER
    
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes=compounding(4.0, 32.0, 1.001)
        # batch up the examples using spaCy's minibatch
                  
        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = nlp.resume_training()

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        for itn in range(iterations):
            random.shuffle(train_data)
            #batches = get_batches(TRAIN_DATA, 'ner') resulted in poor loss
            batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, 
                           annotations,
                           sgd=optimizer, 
                           drop = 0.2, 
                           losses=losses)
            print(f"Losses at iteration {itn} - {dt.datetime.now()} {losses}")
    
    print('Model training completed')
    return nlp

In [12]:
model_params = {
    'model': None,
    'iterations': 30,
    'train_data': TRAIN_DATA,
    'dropout': decaying(0.6, 0.2, 1e-4)
}

In [13]:
nlp = train_model(**model_params)

Created blank 'en' model


/usr/local/lib/python3.6/dist-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs


Losses at iteration 0 - 2020-10-28 22:47:37.331359 {'ner': 1147.6173930254581}
Losses at iteration 1 - 2020-10-28 22:47:57.982143 {'ner': 69.65919949273851}
Losses at iteration 2 - 2020-10-28 22:48:18.257130 {'ner': 90.77040395048208}
Losses at iteration 3 - 2020-10-28 22:48:40.431222 {'ner': 112.9496475175572}
Losses at iteration 4 - 2020-10-28 22:49:03.230598 {'ner': 87.25429809784757}
Losses at iteration 5 - 2020-10-28 22:49:25.961449 {'ner': 66.96094445663779}
Losses at iteration 6 - 2020-10-28 22:49:49.121172 {'ner': 138.27960131744933}
Losses at iteration 7 - 2020-10-28 22:50:12.124257 {'ner': 34.095706850456516}
Losses at iteration 8 - 2020-10-28 22:50:35.106328 {'ner': 51.639370080827156}
Losses at iteration 9 - 2020-10-28 22:50:58.342471 {'ner': 66.8501776199562}
Losses at iteration 10 - 2020-10-28 22:51:24.801616 {'ner': 11.25965027327865}
Losses at iteration 11 - 2020-10-28 22:52:10.123172 {'ner': 20.819562254195294}
Losses at iteration 12 - 2020-10-28 22:52:54.274424 {'ner'

## Test the trained model

In [14]:
nlp.pipeline

[('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f14bd822348>)]

In [18]:

tagger = Tagger(nlp.vocab)
parser = DependencyParser(nlp.vocab)



In [20]:
nlp.add_pipe(parser, before="ner")
nlp.add_pipe(tagger, before="parser")


ValueError: ignored

In [21]:
nlp.pipeline

[('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f14b9b99468>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f14bd822348>)]

In [ ]:
doc = nlp('Here is a green roof on this house. A green roof is good. water piping, I have alot of battery packs')


In [ ]:
displacy.render(doc, style="ent")

## Save model for testing

In [ ]:
output_dir = r'/content/drive/My Drive/Colab Notebooks/nlp-ner-sustain-notebook/model'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

## Loading and testing the saved model

In [ ]:
x = ['i am a green roof']
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)

for text in x:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])